In [7]:
import pandas as pd
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split


# Load your dataset ()
df  = pd.read_csv('../Data/prepared_sales_data.csv')




In [8]:
# Convert categorical variables to numeric
categorical_columns = df.select_dtypes(include=['object']).columns
df = pd.get_dummies(df, columns=categorical_columns)


In [9]:
# Normalize numerical features
scaler = StandardScaler()
numerical_columns = df.select_dtypes(include=['float64', 'int64']).columns
df[numerical_columns] = scaler.fit_transform(df[numerical_columns])


In [10]:
# Split the data
X = df.drop('total_sales', axis=1)
y = df['total_sales']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


: 

In [ ]:
class SalesEnvironment:
    def __init__(self, X, y):
        self.X = X
        self.y = y
        self.reset()
        
    def reset(self):
        self.current_index = 0
        return self.X.iloc[self.current_index].values
    
    def step(self, action):
        actual_sales = self.y.iloc[self.current_index]
        reward = -abs(action - actual_sales)  # Negative absolute error
        
        self.current_index += 1
        done = self.current_index >= len(self.X)
        
        if not done:
            next_state = self.X.iloc[self.current_index].values
        else:
            next_state = None
        
        return next_state, reward, done

In [2]:
# Define the maze
maze = np.array([
    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
    [1, 0, 0, 0, 1, 0, 0, 0, 0, 1],
    [1, 0, 1, 0, 1, 0, 1, 1, 0, 1],
    [1, 0, 1, 0, 0, 0, 0, 0, 0, 1],
    [1, 0, 1, 1, 1, 1, 1, 1, 0, 1],
    [1, 0, 1, 0, 0, 0, 0, 0, 0, 1],
    [1, 0, 0, 0, 1, 1, 1, 1, 0, 1],
    [1, 0, 1, 0, 0, 0, 0, 0, 0, 1],
    [1, 0, 1, 0, 0, 0, 0, 0, 0, 1],
    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
])



In [15]:
df.head()

,code,amount,units,time,province,week,customerId,supermarket_id,basket,day,...,size_oz,size_category,is_bulk,feature,display,postal-code,has_promotion,day_of_week,is_weekend,total_sales
0,7680850106,0.80,1,11:00:00,2,43,125434,244,1,1,...,16.0,LARGE,False,4,5,40222,0,0,0,0.80
1,1800028064,2.25,1,11:37:00,2,43,108320,244,2,1,...,24.0,Unknown Size,Unknown,4,5,40222,0,0,0,2.25
2,9999985067,0.85,1,11:48:00,2,43,162016,244,3,1,...,16.0,LARGE,False,4,5,40222,0,0,0,0.85
3,9999985131,2.19,1,13:23:00,2,43,89437,244,4,1,...,16.0,LARGE,False,4,5,40222,0,0,0,2.19
4,5100002794,2.19,1,13:23:00,2,43,89437,244,4,1,...,26.0,LARGE,False,4,5,40222,0,0,0,2.19


In [3]:
# Define the environment
class MazeEnvironment:
    def __init__(self, maze):
        self.maze = maze
        self.height, self.width = maze.shape
        self.start = (1, 1)
        self.goal = (8, 8)
        self.actions = [(0, 1), (1, 0), (0, -1), (-1, 0)]  # Right, Down, Left, Up
        
    def reset(self):
        return self.start
    
    def step(self, state, action):
        new_state = (state[0] + action[0], state[1] + action[1])
        
        if new_state == self.goal:
            reward = 100
            done = True
        elif self.maze[new_state]:  # Hit a wall
            reward = -10
            new_state = state
            done = False
        else:
            reward = -1
            done = False
        
        return new_state, reward, done


In [17]:
# Initialize the Q-table
env = SupermarketEnv(df)
q_table = np.zeros(env.state_dims + [len(env.actions)])

In [18]:
def choose_action(state, epsilon):
    if np.random.uniform(0, 1) < epsilon:
        return np.random.randint(len(env.actions))  # Explore
    else:
        return np.argmax(q_table[state])  # Exploit

def update_q_table(state, action, reward, next_state):
    q_predict = q_table[state + (action,)]
    q_target = reward + gamma * np.max(q_table[next_state])
    q_table[state + (action,)] += alpha * (q_target - q_predict)


In [19]:
# Train the agent
rewards_history = []
for episode in range(num_episodes):
    state = env.reset()
    done = False
    total_reward = 0
    
    while not done:
        action = choose_action(state, epsilon)
        next_state, reward, done = env.step(action)
        if next_state is None:
            break
        update_q_table(state, action, reward, next_state)
        state = next_state
        total_reward += reward
    
    rewards_history.append(total_reward)
    
    # Decay epsilon
    epsilon = max(epsilon_min, epsilon * epsilon_decay)
    
    if episode % 1000 == 0:
        print(f"Episode: {episode}, Total Reward: {total_reward}, Epsilon: {epsilon:.4f}")

print("Training complete!")

Episode: 0, Total Reward: 5698.932832742482, Epsilon: 0.9950


KeyboardInterrupt: 